In [ ]:
#!!!!注意，請先執行這個區塊載入 Google 雲端硬碟再執行底下的程式
from google.colab import drive
drive.mount('/content/drive') 

#### 10.6.3 遷移學習 (transfer learning)

#### 匯入模組

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### 建立預訓練過的 VGG19 模型物件

In [ ]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

80150528/80134624 [==============================] - 1s 0us/step


#### 凍結 VGGNet 的神經層

In [ ]:
for layer in vgg19.layers:
    layer.trainable = False

#### 添加新的密集層

In [ ]:
# 建立空的模型後, 把載入的 VGG19 模型的神經層加進去：
model = Sequential()
model.add(vgg19)

# 在 VGG19 神經層後面增加負責分類的新神經層：
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### 熱狗堡資料集下載網址 [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home).

In [ ]:
# 可從以上網址下載熱狗堡資料集 (範例已附)
# 將資料集解壓縮後，置於 Google 雲端硬碟的本章資料集內

####使用 ImageDataGenerator 類別

In [ ]:
# 建立兩個 ImageDataGenerator 物件：
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

#### 定義訓練與驗證資料的生成器

In [ ]:
# 執行此程式區塊前，請記得先執行最上面的程式區塊，在 Colab 內啟用 Google 雲端硬碟

# 設定批次量
batch_size=32

# 定義訓練與驗證資料的生成器 

train_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/Colab Notebooks/F1383_Sample/Ch10/10_6_3/hot-dog-not-hot-dog/train',  # directory 請視自己的範例存放路徑彈性修改
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/Colab Notebooks/F1383_Sample/Ch10/10_6_3/hot-dog-not-hot-dog/test',   # directory 請視自己的範例存放路徑彈性修改
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


#### 開始訓練

In [ ]:
#註：由於神經網路的初始權重參數是隨機設定的, 參雜了隨機性, 因此底下 (或您重跑一次) 的結果不會與書中完全一樣, 但模型的能力是相近的
#註：此模型執行時間較長，每週期約需 10 分鐘不等
model.fit(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)

Epoch 1/16
15/15 [==============================] - 615s 42s/step - loss: 0.9482 - accuracy: 0.5815 - val_loss: 0.4983 - val_accuracy: 0.7458
Epoch 2/16
15/15 [==============================] - 596s 41s/step - loss: 0.6107 - accuracy: 0.7167 - val_loss: 0.5022 - val_accuracy: 0.7688
Epoch 3/16
15/15 [==============================] - 619s 43s/step - loss: 0.4097 - accuracy: 0.8176 - val_loss: 0.4980 - val_accuracy: 0.7604
Epoch 4/16
15/15 [==============================] - 619s 43s/step - loss: 0.4489 - accuracy: 0.8026 - val_loss: 0.4784 - val_accuracy: 0.7688
Epoch 5/16
15/15 [==============================] - 618s 43s/step - loss: 0.5017 - accuracy: 0.7639 - val_loss: 0.5724 - val_accuracy: 0.7604
Epoch 6/16
15/15 [==============================] - 607s 42s/step - loss: 0.4071 - accuracy: 0.8125 - val_loss: 0.5122 - val_accuracy: 0.7688
Epoch 7/16
15/15 [==============================] - 620s 43s/step - loss: 0.2864 - accuracy: 0.8712 - val_loss: 0.5204 - val_accuracy: 0.7729
Epoch 